In [25]:
from google.colab import drive,userdata

In [26]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
!pip install python-docx
!pip install PyPDF2

In [30]:
!pip install docx2txt

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=eed208f844f067904fdf43f9c1e94716264557bf5d02c5eb3799a80448789d97
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [31]:
# from docx.opc.exceptions import PackageNotFoundError
import docx2txt
def extract_text(file_path):
    if file_path.endswith(".docx"):
        # Extract text from DOCX file
        return docx2txt.process(file_path)

    elif file_path.endswith(".pdf"):
        # Extract text from PDF file
        text = ""
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page_num in range(len(reader.pages)):
                text += reader.pages[page_num].extract_text()
        return text

    else:
        raise ValueError("Unsupported file type")

In [16]:
# import os
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/Resume/firm-capsule-436804-b5-5f553d9f1043.json"

In [32]:
import os
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.vectorstores.faiss import FAISS
from google.colab import drive
from docx import Document
import google.generativeai as genai
from datetime import datetime
import PyPDF2

api_key_google = userdata.get('google_cloud')
genai.configure(api_key=api_key_google)

# Mount Google Drive
drive.mount('/content/drive')

model = genai.GenerativeModel('gemini-pro')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
def check_relevance_gemini(tailored_resume_path, job_description_path):
    """
    Evaluate the relevance score between a tailored resume and a job description using Gemini Pro.

    Args:
    - tailored_resume_path (str): Path to the tailored resume file.
    - job_description_path (str): Path to the job description file.

    Returns:
    - dict: A dictionary containing the 'score' (float between 0 and 1) and 'reason' (str).
    """
    # Read content from the files
    try:
        resume_text = extract_text(tailored_resume_path)
        job_description = extract_text(job_description_path)
    except Exception as e:
        print(f"Error reading files: {e}")
        return None

    # Define the prompt for Gemini Pro
    prompt = f"""
You are a recruitment expert evaluating how well a tailored resume aligns with a job description. Provide an objective evaluation based on these criteria:
1. Relevance of skills and experience: Do the candidate’s skills, accomplishments, and experience meet the job's core requirements?
2. Domain Match: Are the candidate's experiences and achievements relevant to the industry or role?
3. Clarity and Conciseness: Is the resume well-structured and focused on the job requirements?
4. Highlight gaps or mismatched qualifications realistically.

Only provide an overall score and respond in the following format:
Score: [Score between 0 and 1]
Reason: [A concise explanation of the score]

Here is the tailored resume:
[Resume Start]
{resume_text}
[Resume End]

And the job description:
[Job Description Start]
{job_description}
[Job Description End]
"""

    try:
        # Send the prompt to Gemini Pro
        response = model.generate_content(prompt)
        candidates = response.candidates

        # Check for valid response
        if not candidates or len(candidates) == 0:
            raise ValueError("No candidates found in the response.")

        # Extract content from the first candidate
        content_text = candidates[0].content.parts[0].text
        print(f"Response from Gemini Pro:\n{content_text}")  # Debugging

        # Parse score and reason
        score, reason = None, None
        for line in content_text.split("\n"):
            if line.lower().startswith("score:"):
                try:
                    score = float(line.split(":", 1)[1].strip())
                except ValueError:
                    raise ValueError(f"Invalid score format: {line}")
            elif line.lower().startswith("reason:"):
                reason = line.split(":", 1)[1].strip()

        # Validate parsed data
        if score is None:
            raise ValueError("Failed to extract score from the response.")
        if not reason:
            reason = "No reason provided."

        return {"score": score, "reason": reason}

    except Exception as e:
        print(f"Error in relevance checking: {e}")
        return None


In [39]:
result=check_relevance_gemini("/content/drive/MyDrive/Resume/tailored_resume (8).docx","/content/drive/MyDrive/Resume/Job_description.docx")

Response from Gemini Pro:
Score: 0.4
Reason: While the candidate has a strong background in data analysis and experience in developing database and business intelligence applications, their skills and experiences do not directly align with the job requirements for a Manufacturing Engineer. The resume does not demonstrate knowledge or experience in manufacturing engineering, including providing technical support for manufacturing and quality problems, identifying and implementing cost reduction projects, or executing design and process verification activities. Additionally, the candidate lacks experience in working with manufacturing equipment and completing process validation for new equipment.


In [41]:
if result:
    print(f"Score: {result['score']}")
    print(f"Reason: {result['reason']}")

Score: 0.4
Reason: While the candidate has a strong background in data analysis and experience in developing database and business intelligence applications, their skills and experiences do not directly align with the job requirements for a Manufacturing Engineer. The resume does not demonstrate knowledge or experience in manufacturing engineering, including providing technical support for manufacturing and quality problems, identifying and implementing cost reduction projects, or executing design and process verification activities. Additionally, the candidate lacks experience in working with manufacturing equipment and completing process validation for new equipment.
